#Interpretabilidade do modelo BERT

In [ ]:
!pip install datasets transformers
!pip install lime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 51.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 757.4 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 70.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

data_path = "drive/MyDrive/Mestrado/defesa/data/balanced"
model_path = "drive/MyDrive/Mestrado/defesa/models"

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import lime
import torch
import torch.nn.functional as F
from lime.lime_text import LimeTextExplainer

In [ ]:
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

In [ ]:
from transformers import BertTokenizer, BertModel, AutoModelForSequenceClassification
tokenizer = BertTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased')

model = AutoModelForSequenceClassification.from_pretrained(model_path+"/Best_BERT_WL_balanced_70-15-15.mod")
class_names = labels = ['AcumCargos', 'ConcAdmis', 'CtrlInt', 'FinPubOrc', 'LicContr', 'Nepotis', 'Obras', 'Outros', 'Previd', 'RemunDirDev', 'ServEduc', 'ServSaude', 'TranspAceInf']


In [ ]:
def predictor(texts):
      outputs = model(**tokenizer(texts, return_tensors="pt", padding=True))
      probas = F.softmax(outputs.logits, dim = 1).detach().numpy()
      return probas
 

explainer = LimeTextExplainer(class_names=class_names)

teste=pd.read_csv(data_path+'/comunic_teste.csv')
comunicados = teste.sort_values(by=['label'])

for index, row in comunicados.iterrows():
  texto = row['text'][:2000]
  array = predictor(texto)
  if(row['label']!=array.argmax()):
    print("")
    imp = "**Classe Real: %s &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; Classe Prevista: %s**" % (class_names[row['label']], class_names[array.argmax()])
    printmd(imp)
    #print('Classe Real:',class_names[row['label']], '       Classe Prevista:', class_names[array.argmax()])
    exp = explainer.explain_instance(texto, predictor, num_features=10, num_samples=20)
    exp.show_in_notebook(text=texto)


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
def predictor(texts):
      outputs = model(**tokenizer(texts, return_tensors="pt", padding=True))
      probas = F.softmax(outputs.logits, dim = 1).detach().numpy()
      return probas
 

explainer = LimeTextExplainer(class_names=class_names)

teste=pd.read_csv(data_path+'/comunic_teste.csv')

i = 0
while i<13:
  comunicados = teste[teste.label==i]
  for index, row in comunicados[:2].iterrows():
    texto = row['text'][:2000]
    array = predictor(texto)
    if(row['label']==array.argmax()):
      imp = "**Classe Real: %s &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; Classe Prevista: %s**" % (class_names[row['label']], class_names[array.argmax()])
      printmd(imp)
      exp = explainer.explain_instance(texto, predictor, num_features=10, num_samples=20)
      exp.show_in_notebook(text=texto)
  i=i+1
  


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
'''
def predictor(texts):
    outputs = model(**tokenizer(texts, return_tensors="pt", padding=True))
    probas = F.softmax(outputs.logits, dim = 1).detach().numpy()
    
    return probas

explainer = LimeTextExplainer(class_names=class_names)


teste=pd.read_csv(data_path+'/comunic_teste.csv')
#comunicados=teste[teste.label==1]
comunicados = teste.sort_values(by=['label'])


#for texto in comunicados["text"][:3].values:
for texto in comunicados["text"].values:
    exp = explainer.explain_instance(texto, predictor, num_features=10, num_samples=100)
    exp.show_in_notebook(text=texto)
'''

'\ndef predictor(texts):\n    outputs = model(**tokenizer(texts, return_tensors="pt", padding=True))\n    probas = F.softmax(outputs.logits, dim = 1).detach().numpy()\n    \n    return probas\n\nexplainer = LimeTextExplainer(class_names=class_names)\n\n\nteste=pd.read_csv(data_path+\'/comunic_teste.csv\')\n#comunicados=teste[teste.label==1]\ncomunicados = teste.sort_values(by=[\'label\'])\n\n\n#for texto in comunicados["text"][:3].values:\nfor texto in comunicados["text"].values:\n    exp = explainer.explain_instance(texto, predictor, num_features=10, num_samples=100)\n    exp.show_in_notebook(text=texto)\n'